## 크롤링crawling  
*  웹 상에 존재하는 다양한 컨텐츠를 수집하는 행위
*  html 문서를 통채로 읽어서 내용을 분석한 뒤 필요한 데이터만 골라서 추출하는 것이 주된 작업

## 파이썬으로 크롤링하기
* urllib/urllib2 : 사용하기 무난, 코드 단순, 내장패키지
* requests : 따로 설치, urllib보다 고급기능 제공
* lxml : 웹을 통해 읽어 들인 문서 내용 분석  
         문서내에서 필요한 데이터를 추출하는데 사용

In [1]:
# 관련 패키지 설치 (requests, lxml, cssselect)

In [2]:
import requests
import lxml.html
from lxml.cssselect import CSSSelector

In [3]:
# 한빛미디어 홈페이지(hanb.co.kr)의 베스트셀러 페이지의
# 도서정보(이미지,도서명,저자,가격)을 수집하세요.
url = 'https://www.hanbit.co.kr/store/books/bestseller_list.html'

In [4]:
# 지정한 url의 문서를 읽어옴 : get(주소, 옵선)
res = requests.get(url)

In [5]:
# requests 패키지 실행후 유용한 변수로 결과 확인
# http 응답코드, 응답으로 받은 컨텐츠의 인코딩 및
# 컨텐츠 유형 확인
print(res.status_code, res.encoding,
      res.headers['content-type'])

200 UTF-8 text/html; charset=UTF-8


In [6]:
# 응답으로 받은 컨텐츠 확인 : text, content
# 보통은 text 를 많이씀 ( content 를 사용하는 경우 한글이 깨지는 경우가 생길 수 있음 )
res.text[:500]                # 문자형으로 출력

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>한빛출판네트워크</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛출판네트워크"/>\r\n<meta property="og:description" content="더 넓은 세상, 더 나은 미래를 위'

In [7]:
res.content[:500]             # byte(유니코드)로 출력

b'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>\xed\x95\x9c\xeb\xb9\x9b\xec\xb6\x9c\xed\x8c\x90\xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="\xed\x95\x9c\xeb\xb9\x9b\xec\xb6\x9c\xed\x8c\x90\xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac"/>\r\n<meta property="og:descripti'

In [8]:
# 스크래핑한 결과를 분석해서 필요한 데이터를
# 추출하기 위해 따로 변수로 저장
html = res.text 

In [9]:
# html 문서에 저장된 문서 내 요소들을 탐색하기 편하도록 계층구조DOM로 생성
root = lxml.html.fromstring(html)

In [10]:
# 추출한 제목을 저장하기 위해 리스트 선언
titles = []
authors = []
prices = []
thumbs = []

In [11]:
# 모든 p 태그들 중에서 클래스명이 book_tit인 p태그(들)
# 찾으후 하위 태그가 a인 요소들을 선택한후 
# 텍스트 요소를 추출함
for title in root.cssselect('p.book_tit a'):
    titles.append(title.text_content())
    # print(title.text_content())

In [12]:
for author in root.cssselect('p.book_writer'):
    # print(author.text_content())
    authors.append(author.text_content())

In [13]:
# 도서 가격 추출시 ',' 와 '원'은 제거한후 리스트에 저장 해야함
for price in root.cssselect('span.price'):
    # print(price.text_content())
    prices.append(price.text_content())

In [14]:
for thumb in root.cssselect('img.thumb'):
    # print(thumb.get('src'))
    thumbs.append(thumb.get('src'))

In [15]:
# 추출한 데이터들을 csv 형식 재작성
# 혼자 공부하는 파이썬, 윤인성, 16200, 이미지경로 
data = ''
for i in range(len(titles)):
    print(f'"{titles[i]}","{authors[i]}", {prices[i]}, {thumbs[i]}\n')
    data += f'"{titles[i]}","{authors[i]}", {prices[i]}, {thumbs[i]}\n'


"혼자 공부하는 머신러닝+딥러닝","박해선 ", 23,400원, /data/books/B2002963743_m.jpg

"제대로 알고 쓰는 논문 통계분석 : SPSS & AMOS (개정증보판)","노경섭 ", 28,000원, /data/books/B3589179269_m.jpg

"비겁한 돈","황현희 , 제갈현열 ", 14,400원, /data/books/B7543250605_m.jpg

"365 부모 말하기 연습 일력(스프링북)","박재연 , 공인영 ", 16,200원, /data/books/B5051207171_m.jpg

"1000개 숨은그림찾기 바다 동물","이한이 ", 7,200원, /data/books/B7197264292_m.jpg

"유지보수하기 어렵게 코딩하는 방법: 평생 개발자로 먹고 살 수 있다","로에디 그린 ", 0원, /data/ebook/E2375873090_m.jpg

"회사에서 바로 통하는  실무 엑셀+파워포인트+워드&한글(개정판)","전미진 , 이화진 , 신면철 ", 19,800원, /data/books/B6634694521_m.jpg

"혼자 공부하는 파이썬","윤인성 ", 16,200원, /data/books/B2587075793_m.jpg

"이것이 취업을 위한 코딩 테스트다 with 파이썬","나동빈 ", 30,600원, /data/books/B8945183661_m.jpg

"한번에 통과하는 논문 : SPSS 결과표 작성과 해석 방법","히든그레이스 논문통계팀 ", 23,000원, /data/books/B6318137900_m.jpg

"프리드버그 선형대수학","스티븐 H. 프리드버그 , 아놀드 J. 인셀 , 로렌스 E. 스펜스 ", 35,000원, /data/books/B1710834745_m.jpg

"혼자 공부하는 C 언어","서현우 ", 21,600원, /data/books/B1854329839_m.jpg

"심리 읽어드립니다","김경일 , 사피엔스 스튜디오 ", 15,750원, /data/books/B8953

In [16]:
# 추출한 데이터들을 csv 파일로 저장
hdr = 'title,author,price,thumb\n'
with open('books.csv', 'w') as f:
    f.write(hdr)
    f.write(data)

In [17]:
# 한빛미디어 홈페이지의 새로나온책 페이지의 
# 도서정보(이미지,도서명,저자,가격)를 수집하세요 
# 단, 1~3페이지까지의 도서를 대상으로 추출

In [18]:
# 한빛미디어 홈페이지의 전체도서목록 페이지의 
# 도서정보(브랜드,도서명,저자,발행일,가격)를 수집해보기 

In [19]:
url = 'https://www.hanbit.co.kr/store/books/full_book_list.html'

In [20]:
res = requests.get(url)
html = res.text
root = lxml.html.fromstring(html)

In [21]:
brands = []
titles = []
writers = []
pubdates = []
prices = []

In [22]:
# css선택자가 요소마다 다름
# 이런 경우 이름을 요소로 선택하는 것보다는 
# 요소의 위치를 선택자로 사용해서 요소를 선택하는 것이 좋음 
# for brand in root.cssselect('td.brd_m'): 
# 모든 td태그들 중에서 첫번째 td태그(들)를 선택함 
for brand in root.cssselect('table.tbl_type_list td:first_child'):
    print(brand.text_content())
    #brands.append(brand.get('src'))

한빛미디어
한빛미디어
한빛라이프
한빛비즈
한빛미디어
한빛미디어
한빛에듀
한빛아카데미
한빛아카데미
한빛아카데미
한빛아카데미
한빛아카데미
한빛아카데미
한빛미디어
한빛미디어
한빛미디어
한빛미디어
한빛아카데미
한빛아카데미
한빛라이프
한빛비즈
한빛아카데미
한빛미디어
한빛아카데미
한빛미디어
한빛미디어
한빛미디어
한빛미디어
한빛라이프
한빛비즈
한빛미디어
한빛비즈
한빛라이프
한빛미디어
한빛아카데미
한빛미디어
한빛비즈
한빛라이프
한빛미디어
한빛미디어
한빛비즈
한빛라이프
한빛비즈
한빛미디어
한빛라이프
한빛미디어
한빛에듀
한빛에듀
한빛에듀
한빛미디어


In [23]:
for title in root.cssselect('td.left a'):
    print(title.text_content())
    #.append(thumb.get('src'))

혼자 공부하는 R 데이터 분석
리버스 엔지니어링 기드라 실전 가이드
초등 매일 독서의 힘
저는 주식투자가 처음인데요(기본편 2022년 개정판)
맛있는 디자인 포토샵&일러스트레이터 CC 2022
파워포인트 디자인 실무 강의 with 신프로
1학년 한글 떼기
IT CookBook, C 언어 for Beginner 4판
IT CookBook, 파이썬 for Beginner(3판)
IT CookBook, 엑셀과 파워포인트 2019
IT CookBook, 자바스크립트 프로그래밍 입문(2판)
IT CookBook, 난생처음 C 언어 프로그래밍
IT CookBook, 데이터 과학을 위한 파이썬 머신러닝
SRE를 위한 시스템 설계와 구축
AI로 일하는 기술
이것이 안드로이드다 with 코틀린(3판)
팀장부터 CEO까지 알아야 할 기업 정보보안 가이드
가상현실의 이해
기본 재무관리(제5판)
세밀화로 보는 멸종 동물 도감
저는 주식투자가 처음인데요(투자전략편 2022년 개정판)
(개발자에게 필요 없는 수학은 없다) 프로그래머를 위한 수학
금융 전략을 위한 머신러닝
STEM CookBook, 컴퓨팅 사고력을 키우는 이산수학(3판)
그래프QL 인 액션
비전 시스템을 위한 딥러닝
무던한 개발자를 위한 모던한 자바스크립트
소문난 명강의 : 오준석의 안드로이드 생존코딩_코틀린 편(2판)
하루 한 장 초등과학 365
다시, 배우다
당당한 디자인 결정을 위한 9가지 방법
슈퍼 석세스
365 부모 말하기 연습 일력(스프링북)
엑셀 매크로&VBA 업무 공략집
공학도를 위한 수치해석(8판)
데이터 과학자 되는 법
돈 되는 기획
초등 어휘력이 공부력이다
빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100
스토리텔링 우동이즘의 잘 팔리는 웹툰, 웹소설 이야기 만들기
만화로 배우는 의학의 역사(개정판)
중학교 입학 가이드
비겁한 돈
보고서 작성 실무 강의(개정판)
나도 영문 손글씨 잘 쓰면 소원이 없겠네
구글 BERT의 정석
1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕
1000개 숨은그림

In [24]:
# 동일한 css 선택자가 여러 요소에 사용됨
# for writer in root.cssselect('td.left'):
for writer in root.cssselect('td:nth-child(3)'):
    print(writer.text_content())
    #.append(thumb.get('src'))

강전희 외 1명  
나카지마 쇼타 외 3명  
이은경   
강병욱   
빨간고래(박정아) 외 1명  
신프로(신강식)   
하유정 외 1명  
우재남   
우재남   
김지연   
윤인성   
김화수   
최성철   
헤더 애드킨스 외 5명  
장동인   
고돈호   
강은성   
송은지   
Jonathan Berk 외 2명  
라데크 말리 외 2명  
강병욱   
폴 올랜드   
하리옴 탓샛 외 2명  
박주미   
사메르 부나   
모하메드 엘겐디   
카이 호스트만   
오준석   
치바 가즈요시   
폴 김   
톰 그리버   
댄 페냐   
박재연 외 1명  
최준선   
Steven C.Chapra 외 1명  
에밀리 로빈슨 외 1명  
김도균   
박명선   
한은숙   
우동이즘(이동우)   
장 노엘 파비아니 외 3명  
배혜림   
황현희 외 1명  
홍장표   
윤정희(리제 캘리그라피)   
수다르산 라비찬디란   
전진희   
이한이   
김확실   
발리아파 락슈마난 외 2명  


In [25]:
# 요소에 css 선택자가 없음
for pubdate in root.cssselect('td:nth-child(4)'):
    print(pubdate.text_content())
    #.append(thumb.get('src'))

2022-01-17
2022-01-10
2022-01-10
2022-01-10
2022-01-07
2022-01-05
2022-01-05
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2022-01-03
2021-12-31
2021-12-30
2021-12-30
2021-12-29
2021-12-27
2021-12-26
2021-12-24
2021-12-20
2021-12-20
2021-12-17
2021-12-14
2021-12-10
2021-12-10
2021-12-03
2021-12-01
2021-11-30
2021-11-30
2021-11-26
2021-11-26
2021-11-25
2021-11-25
2021-11-15
2021-11-15
2021-11-11
2021-11-08
2021-11-08
2021-11-05
2021-11-03
2021-11-02
2021-11-02
2021-11-02
2021-11-01


In [26]:
# 코드가 없는 경우 하위 설정 할때 + 로 해도됨 ( 호연씨 코드 ) 
for pubdate in root.cssselect('td.left+td.left+td'):
    pubdates.append(pubdate.text_content())
print(pubdates)

['2022-01-17', '2022-01-10', '2022-01-10', '2022-01-10', '2022-01-07', '2022-01-05', '2022-01-05', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2022-01-03', '2021-12-31', '2021-12-30', '2021-12-30', '2021-12-29', '2021-12-27', '2021-12-26', '2021-12-24', '2021-12-20', '2021-12-20', '2021-12-17', '2021-12-14', '2021-12-10', '2021-12-10', '2021-12-03', '2021-12-01', '2021-11-30', '2021-11-30', '2021-11-26', '2021-11-26', '2021-11-25', '2021-11-25', '2021-11-15', '2021-11-15', '2021-11-11', '2021-11-08', '2021-11-08', '2021-11-05', '2021-11-03', '2021-11-02', '2021-11-02', '2021-11-02', '2021-11-01']


In [27]:
# for price in root.cssselect('td.right'):
for price in root.cssselect('td:last-child'):
    prices.append(price.text_content())
print(prices)
    #.append(thumb.get('src'))

['\n                \n                \n                {{ bookSummary.author }} 지음\n                {{ bookSummary.publishingCompany }} | {{ bookSummary.date }}\n                \n                \n                {{ bookSummary.bookType }}\n                \n              ', '22,000원', '50,000원', '17,000원', '18,500원', '22,000원', '18,000원', '12,000원', '26,000원', '25,000원', '23,000원', '27,000원', '25,000원', '30,000원', '40,000원', '18,000원', '34,000원', '25,000원', '22,000원', '42,000원', '22,000원', '18,500원', '42,000원', '35,000원', '29,000원', '35,000원', '38,000원', '32,000원', '32,000원', '19,800원', '16,000원', '22,000원', '25,000원', '18,000원', '22,000원', '36,000원', '24,000원', '14,500원', '18,000원', '18,000원', '16,000원', '20,800원', '16,500원', '16,000원', '18,000원', '16,800원', '34,000원', '8,000원', '8,000원', '8,000원', '38,000원']


### 네이버 검색예제
* search.naver.com/search.naver?query=검색어
* 네이버에 검색어를 입력하고 검색결과에서 데이터 추출
* 질의문자열 querystring 을 이용해서 검색하고 그 결과에서 필요한 데이터를 추출함
* 특정 사이트는 스크래핑이나 크롤링을 막기 위한 방편으로 사이트에 접속하는 사용자의 useragent를 확인함
* UA없이 사이트 접속을 시도하면 접속권한 거부의 의미로 403 응답코드와 함께 접속을 금지하기도 함 

In [28]:
# 특정 사이트는 스크래핑이나 크롤링을 막기 위한 방편으로
# 사이트에 접속하는 사용자의 useragent를 확인함
# UA없이 사이트 접속을 시도하면 접속권한 거부의 의미로
# 403 응답코드와 함께 접속을 금지하기도 함 

In [29]:
query = 'html5' #검색어
url = 'https://search.naver.com/search.naver'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
params = {'query' : query}

res = requests.get(url, headers=headers, params=params)
html = res.text
html[:500]
root = lxml.html.fromstring(html)

In [30]:
# 블로그, 웹사이트, 네이버책, 포스트 등의
# 제목과 링크 추출

In [31]:
titles = []
links = []

In [32]:
for web in root.cssselect('panel-list a.api_txt_lines'):
    print(web.text_content())
    print(web.get('href'))

Html5 구조 | 기초부터 디자인까지 셀프코딩#2
https://blog.naver.com/call_1994/222253172378
[html5 &CSS] 박스 모델 - round/ border radius 모서리 곡률(경계선 곡률)#046
https://blog.naver.com/kji9653/222602416577
HTML5/에디터/아톰/Atom/실시간 확인
https://blog.naver.com/scyan2011/222596844291
html+,css+랑 html5+,css3 차이점이 뭐에요???
https://cafe.naver.com/javachobostudy/177887
html5 공간 분할 태그
https://blog.naver.com/suyeoniiii/222609250284
설치 없이 바로 즐기는 요즘 대세HTML5 게임 5
https://post.naver.com/viewer/postView.naver?volumeNo=32900007&memberNo=15460786&vType=VERTICAL
html에서 이미지 첨부 및 오디오 첨부를 했음에도 불구하고 실행이 되지 않을 경우에 대해 질문드립니다.
https://cafe.naver.com/hacosa/303083


In [33]:
for brain in root.cssselect('a.question_text'):
    print(brain.text_content())
    print(brain.get('href'))

Html5 자바스크립트 select와 audio연결방법이 궁금합니다! [내공 100] 안녕하세요. 제가 코드를 하나 짜고있는데, select의 value 값들로 들어가있는 mp3노래 파일들을 audio.src 객체 파일로 가져오려면 어떻게 해야할까요..? 다른 부분의 소스 코드들도 짜보긴했지만 잘 짜여진 것인지 모르겠습니다.. 제 코드는 사진... 
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040205&docId=406193296&qb=aHRtbDU=&enc=utf8&section=kin.ext&rank=1&search_sort=0&spq=0
html5 css3 인강 추천 좀 html5css3공부하려구요 웹페이지 만들어보려는데 책은 어려워서요 단기간 준비할 수 있는 곳으로 추천좀요
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10501&docId=384782269&qb=aHRtbDU=&enc=utf8&section=kin.ext&rank=2&search_sort=0&spq=0
CSS,HTML 과 CSS3,HTML5의 차이점이 뭔가요? (간단히 요약해주세요!)
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040205&docId=379944783&qb=aHRtbDU=&enc=utf8&section=kin.ext&rank=3&search_sort=0&spq=0


In [34]:
for wiki in root.cssselect('a.link_tit'):
    print(wiki.text_content())
    print(wiki.get('href'))

 HTML5 - 나무위키 
https://namu.wiki/w/HTML5
 W3C HTML 
https://www.w3.org/html/
 HTML5test - How well does your browser support HTML5? 
https://html5test.com/
 Web Hypertext Application Technology Working Group (WHATWG) 
https://whatwg.org/


In [35]:
for book in root.cssselect('a.title_area'):
    print(book.text_content())
    print(book.get('href'))

In [36]:
# jtbc 뉴스 사이트에서
# 사진npic, 제목title, 기사미리보기preview, 범주ntype 등을
# 크롤링해서 jtbcnews 테이블에 저장
# 단, 뉴스기사는 2020-10-22 일자를 대상으로 한다 

In [37]:
url = 'https://news.jtbc.joins.com/section/list.aspx'
params = {'pdate':'20201022'}

In [38]:
res = requests.get(url, headers=headers, params=params)

In [39]:
html = res.text
root = lxml.html.fromstring(html)

In [40]:
res.text[:500]

'\n<!DOCTYPE html> \n    <!--[if IE 7]><html lang="ko" class="ie7"><![endif]--> \n    <!--[if IE 8]><html lang="ko" class="ie8"><![endif]--> \n    <!--[if IE 9]><html lang="ko" class="ie9"><![endif]--> \n    <!--[if gt IE 9]><!--> <html lang="ko"> <!--<![endif]--> \n    <head id="Head1"><script type="text/javascript">document.domain="joins.com";</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="cache-control" content="no-cache" />\r\n    <Meta name="title" con'

In [41]:
npics = []
titles = []
previews = []
ntypes = []

In [42]:
for npic in root.cssselect('dd.photo a img'):
    print(npic.get('src'))
    npics.append(npic.get('src'))

https://photo.jtbc.joins.com/news/2020/10/22/202010222242142756_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222217403478_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222102401754_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222120070095_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010220930056459.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010220927336004.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222114196734_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222113384343_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222108560614_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/26/202010261603123770_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222110571415_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/2020/10/22/202010222107187996_LC.jpg.tn350.jpg
https://photo.jtbc.joins.com/news/

In [43]:
for title in root.cssselect('dt.title_cr a'):
    print(title.text_content())
    titles.append(title.text_content())

건설 폐기물 파묻은 땅에서 벼농사…'브랜드 쌀'로 판매
[단독] '술접대' 지목 유흥업소…"김봉현과 검사들 왔었다"
[원보가중계] 국감 중에 이러시면…강훈식, 또 게임하다 딱 걸려
[날씨박사] 먼지에 한파까지…23일 아침은 더 춥다
둥근 공의 '심술'…챔피언스리그 흔드는 '자책골'
김태균 '눈물의 은퇴'…"20년간 우승 못 해서 아쉬워"
[팩트체크] 월성 원전 방사성물질이 후쿠시마 130배?
'채용비리 혐의' LG전자 전·현직 임직원 12명 검찰 송치
구글, 게임업체에 '구글 규제법' 반대 종용 의혹
"택배노동자 사망, 업무와 무관" 쿠팡 측 주장 따져보니…
택배노동자 또 숨져…"한 번 출근하면 2,3일씩 근무"
총선 토론회서 '세월호 유가족에 막말' 차명진 기소
"나경원 아들, 엄마 찬스"…서울대 국감서 도마 올라
[밀착카메라] 13살도 전동킥보드 허용…안전은 누가?
윤석열, 문 대통령 메시지 공개…청와대 "입장 없다"


In [44]:
for preview in root.cssselect('dd.read_cr a'):
    print(preview.text_content())
    previews.append(preview.text_content())

 [앵커]경기도 일부 지역에서 '건설 폐기물'이 섞인 논에서 자란 벼를 브랜드 쌀로 포장해 팔고 있습니다. 저희 취재진이 다섯 달 동안 현장을 추적했는데요. 업체들이 처리 비용을 아끼려고 논에다 폐기물을 퍼부었...
 [앵커]오늘(22일) 국정감사에선 김봉현 전 회장이 주장한 '검사 술접대 의혹'도 쟁점이 됐습니다. 아직 그 진위는 확인되지 않고 있는데, 일단 당사자 측은 강하게 반박하고 있습니다. 저희 법조팀이 그 유흥업소를...
 좋은 밤, 좋은 뉴스 < 원보가중계 > 시작합니다.첫 번째 소식입니다.앞서 대검찰청 국정감사 리포트에서 미처 다루지 못한 소식이 있어서 준비했습니다.열린민주당 김진애 의원, 윤석열 검찰총장 상대로 부인의 재...
 [앵커]날씨박사 시간입니다. 기상학 박사, 김세현 기상전문기자가 나와 있습니다.김세현 기자, 오늘(22일) 황사 때문에 전국 곳곳에 미세먼지 주의보가 내려졌고, 지금도 내려진 곳이 있다고 하는데, 오늘 밤엔 한...
[앵커]축구에서 자책골은 골문을 지키는 선수라면 어떻게든 피하고 싶은 순간이죠. 하지만 의도치 않은 순간, 야속한 장면이 튀어나오곤 합니다. 엉뚱한 곳에 맞아서 골라인을 넘고, 또 막아냈다 싶다가도 골이 되는...
[김태균/한화이글스 : 한화이글스는 저의 자존심이었고 자부심이었습니다.][앵커]20년 선수 생활을 내려놓으면서 한화의 4번 타자 김태균은 끝내 울어버렸습니다. 홈런만 대접받던 우리 야구에 출루의 가치를 알려준...
 [앵커]일본 정부가 '방사능 오염수'를 바다에 흘려보낼 방침을 이달 안에 확정할 것으로 보입니다. 일본 정치권에서는 "한국의 원전에서 나오는 방사성 물질이 후쿠시마의 130배다"라는 주장이 나왔습니다. 저희 팩...
 경찰이 LG전자 전·현직 임직원 12명을 채용비리 혐의로 검찰에 넘겼습니다.지난 2013년에서 2015년 사이, LG전자 한국영업본부 공개채용 과정에서 채용 청탁이 들어온 지원자들을 부당하게 합격시킨 혐의입...
 [앵커]최근 미국 정부가 구글을 상대로 반독점법 위반 혐

In [45]:
for ntype in root.cssselect('span.location'):
    ntype = ntype.text_content().replace('\t', '')
    ntype = ntype.replace('\r\n', '')
    ntype = ntype.replace(' ', '')
    ntype = ntype.replace('[JTBC>', '')
    ntype = ntype.replace(']', '')
    ntypes.append(ntype)

In [46]:
ntypes

['사회',
 '사회',
 '정치',
 '날씨',
 '스포츠',
 '스포츠',
 '사회',
 '사회',
 '정치',
 '사회',
 '사회',
 '사회',
 '정치',
 '사회',
 '정치']

In [47]:
#re 사용해서 하는 내용물 전처리 
import re 
for ntype in root.cssselect('span.location'):
    ntype = re.sub('\t|\r\n|','',ntype.text_content())
    ntype = re.sub('\[JTBC>|\]', '', ntype)
    ntypes.append(ntype)

In [48]:
# 추출한 데이터들을 csv 형식 재작성 
data = ''
for i in range(len(titles)):
    rows = f'{titles[i]}{previews[i]}{ntypes[i]}{npics[i]}\n'
    print(rows)
    
print(len(titles), len(previews), len(ntypes), len(npics))

건설 폐기물 파묻은 땅에서 벼농사…'브랜드 쌀'로 판매 [앵커]경기도 일부 지역에서 '건설 폐기물'이 섞인 논에서 자란 벼를 브랜드 쌀로 포장해 팔고 있습니다. 저희 취재진이 다섯 달 동안 현장을 추적했는데요. 업체들이 처리 비용을 아끼려고 논에다 폐기물을 퍼부었...사회https://photo.jtbc.joins.com/news/2020/10/22/202010222242142756_LC.jpg.tn350.jpg

[단독] '술접대' 지목 유흥업소…"김봉현과 검사들 왔었다" [앵커]오늘(22일) 국정감사에선 김봉현 전 회장이 주장한 '검사 술접대 의혹'도 쟁점이 됐습니다. 아직 그 진위는 확인되지 않고 있는데, 일단 당사자 측은 강하게 반박하고 있습니다. 저희 법조팀이 그 유흥업소를...사회https://photo.jtbc.joins.com/news/2020/10/22/202010222217403478_LC.jpg.tn350.jpg

[원보가중계] 국감 중에 이러시면…강훈식, 또 게임하다 딱 걸려 좋은 밤, 좋은 뉴스 < 원보가중계 > 시작합니다.첫 번째 소식입니다.앞서 대검찰청 국정감사 리포트에서 미처 다루지 못한 소식이 있어서 준비했습니다.열린민주당 김진애 의원, 윤석열 검찰총장 상대로 부인의 재...정치https://photo.jtbc.joins.com/news/2020/10/22/202010222102401754_LC.jpg.tn350.jpg

[날씨박사] 먼지에 한파까지…23일 아침은 더 춥다 [앵커]날씨박사 시간입니다. 기상학 박사, 김세현 기상전문기자가 나와 있습니다.김세현 기자, 오늘(22일) 황사 때문에 전국 곳곳에 미세먼지 주의보가 내려졌고, 지금도 내려진 곳이 있다고 하는데, 오늘 밤엔 한...날씨https://photo.jtbc.joins.com/news/2020/10/22/202010222120070095_LC.jpg.tn350.jpg

둥근 공의 '심술'…챔피언스리그 흔드는 '자책골'[앵커]축구에서 자책골은 골문을 지키는 선수라면 어떻